In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=279bad4479e520c27586173dc19f9cf40dfac62f0fe75eb22da4b838edcd71f5
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    try:
      coverpage= soup.find_all('header',class_='node-header')
      title=coverpage[0].find('h1').get_text()
      # print(title)
    except:
      title=''

    
    try:   
      coverpage = soup.find_all('div', class_='date-publish')
      time = coverpage[0].get_text()    
      # print(time)
    except:
      time=''

    try:
      coverpage = soup.find_all('div', class_='field-items')
      article=''
      for j in range(0,len(coverpage)):
        for i in coverpage[j].find_all('p'):
          article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('error\n'+url)
  except:
    title=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,time,article]

In [35]:
get_articles('https://www.lostiempos.com/actualidad/mundo/20220611/colombia-campana-electoral-se-pone-al-rojo-vivo')

['\n          Colombia: la campaña electoral se pone al rojo vivo        ',
 '\n            Publicado el 11/06/2022 a las 8h21    ',
 '\nBOGOTÁ, Efe\n\tLos videos sobre las discusiones internas de la campaña del izquierdista Gustavo Petro y una nueva encuesta de intención de voto pusieron ayer al rojo vivo la campaña presidencial colombiana a medida que se acerca la cita con las urnas.\n\tUn sondeo de Invamer para El Espectador, Noticias Caracol y Blu Radio, publicado ayer, muestra una apretada lucha entre los dos aspirantes a la presidencia de Colombia.\n\tEn la intención de voto Rodolfo Hernández, un empresario de la construcción millonario que concurre por la Liga de Gobernantes Anticorrupción, logra un 48,2%, mientras que Petro, del Pacto Histórico, obtiene 47,2%.\nÚltima encuesta\n\tLa encuesta cobra relevancia porque es una de las últimas que saldrá a la luz pública, ya que a partir de mañana la ley no permite publicarlas.\n\tEn ese estudio el margen de error es del 2,69%, lo que

In [36]:
pandarallel.initialize(nb_workers= 10, progress_bar=True)

df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/colab/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
output=df['rows'].parallel_apply(get_articles)

df['country']='Bolivia'
df['title']=''
df['time']=''
df['article']=''
for i in range(0,len(df)):
  df.loc[i]['title','time','article']=output[i]


df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
name_article='/content/drive/MyDrive/colab/url_article.csv'
df.to_csv(name_article, index=False)
print(df)


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


                                                  page  \
0    https://www.lostiempos.com/actualidad/mundo?pa...   
1    https://www.lostiempos.com/actualidad/mundo?pa...   
2    https://www.lostiempos.com/actualidad/mundo?pa...   
3    https://www.lostiempos.com/actualidad/mundo?pa...   
4    https://www.lostiempos.com/actualidad/mundo?pa...   
..                                                 ...   
621  https://www.lostiempos.com/actualidad/mundo?pa...   
622  https://www.lostiempos.com/actualidad/mundo?pa...   
623  https://www.lostiempos.com/actualidad/mundo?pa...   
624  https://www.lostiempos.com/actualidad/mundo?pa...   
625  https://www.lostiempos.com/actualidad/mundo?pa...   

                                                  rows  country  \
0    https://www.lostiempos.com/actualidad/mundo/20...  Bolivia   
1    https://www.lostiempos.com/actualidad/mundo/20...  Bolivia   
2    https://www.lostiempos.com/actualidad/mundo/20...  Bolivia   
3    https://www.lostiempos.com/act